Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart Kernel) and then **run all cells** (in the menubar, select Run$\rightarrow$Run All Cells). Alternatively, you can use the **validate** button in the assignment list panel.

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE". When you insert your Code you can remove the line `raise NotImplementedError()`. Also put your name, matriculationnumber, and collaborators below:

In [ ]:
NAME = ""
MATRICULATIONNUMBER = ""
COLLABORATORS = ""

---

<img src="images/logo_ifn.svg" alt="Drawing" style="width: 256px;" align="right"/>

# Exercise 2.3: Machine Learning on MNIST

After we solved two toy problems on artificial 2D data distributions, we will now approach a first more realistic problem, the digit classification task on MNIST. The dataset is well-known in the machine learning community and a lot of fundamental machine learning research is still carried out on this dataset. To avoid extensive preprocessing, we will use the prepared version of this dataset from the torchvision library. 

In [11]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn import neural_network
from torchvision.datasets import MNIST
import params
from ipylab import JupyterFrontEnd
from threadpoolctl import threadpool_limits

PARAM_1, PARAM_2, PARAM_3 = params.gen_params(os.getcwd())
PARAM_4 = int(params.gen_params(os.getcwd(), mode='float', num=1)[0] *100000)
app = JupyterFrontEnd()
app.commands.execute('notebook:render-all-markdown')

### Task 2.3-A: MNIST Data Loading (5P) 

Before training models on MNIST, we need to normalize the data and reduce the dataset size slightly to enable training in a decent amount of time. The data from torchvision is already normalized and converted to a numpy 1D array. After this, implement the following functions:
- Implement the funtion `filter_classes` which filters the data according to the specified digit classes. Assume that the tuple `classes` can have an arbitrary length. 
- Implement the function `shuffle_data` which shuffles the data. You can use whichever function you like for the shuffling (we will not test the exact implementation).
- Implement the function `reduce_set` which only keeps `nb_samples_per_class` samples per class and discards the other samples. Assume that the number of classes in the input is not known in advance.
- Implement the function `split_tr_val` which splits the data such that a fraction `proportion_train` of the data is kept in the training set and the rest is put in the validation set.
- Implement the function `load_tr_val_splits`, which first filters the data according to the parameter `classes`, then shuffles the data with the random seed {{PARAM_4}}, keeps `nb_samples_per_class` samples per class, shuffles the data again with a random seed {{PARAM_4*2}}, and splits the data in a fixed ratio of 70\% training data and 30\% validation data.
- Load the data with `X_tr, Y_tr, X_val, Y_val = load_tr_val_splits(classes=(3,8), nb_samples_per_class=1000)` as given below.


In [ ]:
def load_data(sample_id = 8):
    mntrain = MNIST('/work/datasets', train=True)
    print("The label is: " + str(mntrain.targets.numpy()[sample_id]))
    plt.imshow(mntrain.data.numpy()[sample_id])
    plt.show()
    return mntrain
mntrain = load_data(33)

In [ ]:
X, Y = mntrain.data.flatten(1).float().numpy()/255.0, mntrain.targets.numpy()

def filter_classes(X, Y, classes=(3, 8)):
    # YOUR CODE HERE
    mask = np.isin(Y, classes)
    X_N, Y_N = X[mask], Y[mask]
    return X_N, Y_N

X, Y = filter_classes(X, Y)

def shuffle_data(X, Y, seed=0):
    np.random.seed(seed)
    # YOUR CODE HERE
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    X, Y = X[indices], Y[indices]
    return X, Y

X, Y = shuffle_data(X, Y)

def reduce_set(X, Y, nb_samples_per_class=500):
    # YOUR CODE HERE
    unique_classes = np.unique(Y)
    X_N, Y_N = [], []
    for cls in unique_classes:
        indices = np.where(Y == cls)[0][:nb_samples_per_class]
        X_N.extend(X[indices])
        Y_N.extend(Y[indices])
    X_N, Y_N = np.array(X_N), np.array(Y_N)
    return X_N, Y_N

X, Y = reduce_set(X, Y)

def split_tr_val(X, Y, proportion_train=0.7):
    # YOUR CODE HERE
    num_samples = len(X)
    num_train = int(num_samples * proportion_train)
    X_tr, Y_tr = X[:num_train], Y[:num_train]
    X_val, Y_val = X[num_train:], Y[num_train:]
    return X_tr, Y_tr, X_val, Y_val

X_tr, Y_tr, X_val, Y_val = split_tr_val(X, Y)

def load_tr_val_splits(classes, nb_samples_per_class):
    mntrain = MNIST('/work/datasets', train=True)
    X, Y = mntrain.data.flatten(1).float().numpy()/255.0, mntrain.targets.numpy()
    # YOUR CODE HERE
    X, Y = filter_classes(X, Y, classes)
    X, Y = shuffle_data(X, Y, seed=13832)
    X, Y = reduce_set(X, Y, nb_samples_per_class)
    X_tr, Y_tr, X_val, Y_val = split_tr_val(X, Y)
    return X_tr, Y_tr, X_val, Y_val
    
X_tr, Y_tr, X_val, Y_val = load_tr_val_splits(classes=(3,8), nb_samples_per_class=1000)

In [ ]:
assert type(X_tr) == np.ndarray
assert type(Y_tr) == np.ndarray
assert type(X_val) == np.ndarray
assert type(Y_val) == np.ndarray


### Task 2.3-B: Support Vector Machines (5P) 

For comparison with a deep neural network, we want to write a function `train_svm` that takes the training data as input and returns a trained model:
- Implement the corresponding function using the sklearn library. Make sure to initialize the random_state variable with {{PARAM_4}} to ensure reproducibility of your trainings.
- Train a model `clf_svm1` with the default parameters given in the function definition. What do you observe regarding the training and validation accuracy? 
- Train a second model `clf_svm2` with which you improve the performance on the validation set (no specific performance improvement is demanded).

In [ ]:
def train_svm(X_tr, Y_tr, 
              penalty = 1, 
              kernel_function = 'rbf',
              poly_degree = 1,
              gamma = 10,
              zero_coeff = 0.0,
              stopping_criterion = 1e-3,
              max_iterations = -1):
    # YOUR CODE HERE
    clf = svm.SVC(C=penalty, kernel=kernel_function, degree=poly_degree, gamma=gamma, coef0=zero_coeff,
                  tol=stopping_criterion, max_iter=max_iterations, random_state=13832)
    clf.fit(X_tr, Y_tr)
    return clf

with threadpool_limits(limits=1, user_api='blas'):
    # YOUR CODE HERE
    clf_svm1 = train_svm(X_tr, Y_tr)

    clf_svm2 = train_svm(X_tr, Y_tr, gamma=0.1, C=0.5)

In [ ]:
assert type(clf_svm1) == svm._classes.SVC
assert type(clf_svm2) == svm._classes.SVC
training_accuracy1 = clf_svm1.score(X_tr, Y_tr)
validation_accuracy1 = clf_svm1.score(X_val, Y_val)
training_accuracy2 = clf_svm2.score(X_tr, Y_tr)
validation_accuracy2 = clf_svm2.score(X_val, Y_val)


### Task 2.3-C: Multi-Layer Neural Networks (5P) 

In the subsequent task the aim is to implement a function `train_nnet` that takes the training data as input, trains a neural network and returns the trained model. Implement the following:
- Implement the corresponding function using the sklearn library. Make sure to initialize the random_state variable with {{PARAM_4}} to ensure reproducibility of your trainings.
- Train a model `clf_nnet1` with the default parameters given in the function definition. What do you observe regarding the training and validation accuracy? 
- Train a second model `clf_nnet2` with which you improve the performance on the validation set. How do different parameters affect the result? What do you observe when optimizing hyperparameters and why?

In [ ]:
def train_nnet(X_tr, Y_tr,
               hidden_layer_sizes = (100, 100),
               activation = "relu", 
               solver = "sgd",
               alpha = 0.0001,
               batch_size = 200, 
               learning_rate = "constant",
               learning_rate_init = 0.001,
               max_iter = 50,
               shuffle = True,
               momentum = 0.9,
               nesterovs_momentum = True):
    # YOUR CODE HERE
    clf = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes,
                        activation=activation,
                        solver=solver,
                        alpha=alpha,
                        batch_size=batch_size,
                        learning_rate=learning_rate,
                        learning_rate_init=learning_rate_init,
                        max_iter=max_iter,
                        shuffle=shuffle,
                        momentum=momentum,
                        nesterovs_momentum=nesterovs_momentum,
                        random_state=13832)
    clf.fit(X_tr, Y_tr)
    return clf

with threadpool_limits(limits=1, user_api='blas'):
    # YOUR CODE HERE
    clf_nnet1 = train_nnet(X_tr, Y_tr)
    clf_nnet2 = train_nnet(X_tr, Y_tr, hidden_layer_sizes=(50, 50), learning_rate_init=0.01, max_iter=100)


In [ ]:
assert type(clf_nnet1) == neural_network._multilayer_perceptron.MLPClassifier
assert type(clf_nnet2) == neural_network._multilayer_perceptron.MLPClassifier
training_accuracy1 = clf_nnet1.score(X_tr, Y_tr)
validation_accuracy1 = clf_nnet1.score(X_val, Y_val)
training_accuracy2 = clf_nnet2.score(X_tr, Y_tr)
validation_accuracy2 = clf_nnet2.score(X_val, Y_val)


### Task 2.3-D: Evaluation of the Trained Models (5P) 

When training support vector machines and neural networks, you have probably noticed that both achieve near 100\% accuracy on the training set and a strong performance on the validation set. Still, we optimized hyperparameters for the validation set and it is not yet clear, how the models perform on completely unknown data. For this purpose, we will now employ our test set:
- Implement the below function to load all (official!) test samples of MNIST. Have a look at the `MNIST` class to determine how the test samples can be loaded. Filter the test set such that it only contains the digit classes specified by `classes`. 
- Load the test data into variable `X_test` and `Y_test`. Evaluate your models on the test set. Discuss which one performs better and why.

In [ ]:
def load_test_splits(classes):
    # YOUR CODE HERE
    mnist_test = MNIST('/work/datasets', train=False, download=True)
    X_test, Y_test = mnist_test.data.flatten(1).float().numpy()/255.0, mnist_test.targets.numpy()
    X_test, Y_test = filter_classes(X_test, Y_test, classes)
    return X, Y

with threadpool_limits(limits=1, user_api='blas'):
    # YOUR CODE HERE
    X_test, Y_test = load_test_splits(classes=(3,8))
    
test_accuracy_svm1 = clf_svm1.score(X_test, Y_test)
test_accuracy_svm2 = clf_svm2.score(X_test, Y_test)

test_accuracy_nnet1 = clf_nnet1.score(X_test, Y_test)
test_accuracy_nnet2 = clf_nnet2.score(X_test, Y_test)

In [ ]:
assert type(X_test) == np.ndarray
assert type(Y_test) == np.ndarray
assert type(clf_svm1) == svm._classes.SVC
assert type(clf_svm2) == svm._classes.SVC
assert type(clf_nnet1) == neural_network._multilayer_perceptron.MLPClassifier
assert type(clf_nnet2) == neural_network._multilayer_perceptron.MLPClassifier


### Task 2.3-E: Classification of Different Digits (5P) 

While we previously only considered the digit classes 3 and 8, we now want to see how our models perform for other digit classes. For this purpose complete the function `eval_other_digits` below:
- Load training data with the digit classes specified by `classes` using your previously implemented `load_tr_val_splits` with `nb_samples_per_class=1000`. Remember that this function needs to be deterministic.
- train an SVM and a neural network on this data using `train_svm` and `train_nnet` functions using the default parameters. Calculate the accuracies on the test set for both models and return them as `test_svm_acc` and `test_nnet_acc`.
- Try out how the models perform for different class combinations and also for more than 2 classes and discuss the result with your fellow students. 

In [ ]:
def eval_other_digits(classes):
    # YOUR CODE HERE
    X_tr, Y_tr, _, _ = load_tr_val_splits(classes, nb_samples_per_class=1000)
    
    clf_svm = train_svm(X_tr, Y_tr)
    clf_nnet = train_nnet(X_tr, Y_tr)
    
    X_test, Y_test = load_test_splits(classes)
    
    test_svm_acc = clf_svm.score(X_test, Y_test)
    test_nnet_acc = clf_nnet.score(X_test, Y_test)

    return test_svm_acc, test_nnet_acc

with threadpool_limits(limits=1, user_api='blas'):
    test_svm_acc, test_nnet_acc = eval_other_digits((1,7))
    print('Result svm-Testing - Accuracy on test data = ' + '{0:g}%'.format(test_svm_acc * 100) + "\n")
    print('Result nnet-Testing - Accuracy on test data = ' + '{0:g}%'.format(test_nnet_acc * 100) + "\n")

In [ ]:
assert test_svm_acc >= 0 and test_svm_acc <= 1
assert test_nnet_acc >= 0 and test_svm_acc <= 1


### Task 2.3-F: Differing Data Amounts (5P) 

Finally, we want to gain insights into the performances of both neural networks and SVMs on different data amounts. For this purpose complete the function `eval_data_amount` given below:
- Load training data with `classes=(3,8)` using your previously implemented `load_tr_val_splits` with `nb_samples_per_class` as specified by `nb_samples`. Remember that this function needs to be deterministic.
- train an SVM and a neural network on this data using `train_svm` and `train_nnet` functions using the default parameters. Calculate the accuracies on the test set for both models and return them as `test_svm_acc` and `test_nnet_acc`.
- Try out how the models perform for different amounts of data and discuss the result with your fellow students. 

In [ ]:
def eval_data_amount(nb_samples):
    # YOUR CODE HERE
    X_tr, Y_tr, _, _ = load_tr_val_splits(classes=(3, 8), nb_samples_per_class=nb_samples)
    
    clf_svm = train_svm(X_tr, Y_tr)
    clf_nnet = train_nnet(X_tr, Y_tr)
    
    X_test, Y_test = load_test_splits(classes=(3, 8))
    test_svm_acc = clf_svm.score(X_test, Y_test)
    test_nnet_acc = clf_nnet.score(X_test, Y_test)
    
    return test_svm_acc, test_nnet_acc

with threadpool_limits(limits=1, user_api='blas'):
    test_svm_acc, test_nnet_acc = eval_data_amount(1000)
    print('Result svm-Testing - Accuracy on test data = ' + '{0:g}%'.format(test_svm_acc * 100) + "\n")
    print('Result nnet-Testing - Accuracy on test data = ' + '{0:g}%'.format(test_nnet_acc * 100) + "\n")

In [ ]:
assert test_svm_acc >= 0 and test_svm_acc <= 1
assert test_nnet_acc >= 0 and test_svm_acc <= 1
